In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import json
from collections import namedtuple
import csv
import glob
from time import strptime

In [3]:
def scrape_table(year, week, soup, headers_=False):
    teams = []
    for i, team in enumerate(soup.table.find_all('tr')[1:]):
        if not i:
            headers = ["YEAR", "WEEK"]+[t.text for t in team.find_all('td')]+["GAMES"]
        else:
            games = team.find(class_ = 'previous-games').text.replace(u'\xa0', u' ').encode('utf-8', 'replace')
            teams.append([year, week, i]+[t.text for t in team.find_all('li')] + [games.split("This Week:")[-1]])
    if headers_:
        return headers, teams
    else:
        return teams

In [10]:
def scrape_AP_Top_25_ESPN(year = 2016, path = 'NCAA_AP_TOP_25.csv', nice=True):
    week = 1
    first = True
    while True:
        if nice:
            time.sleep(1)
        url = 'http://espn.go.com/mens-college-basketball/rankings/_/poll/1/year/'+str(year)+'/week/'+str(week)+'/seasontype/2'
        soup = BeautifulSoup(requests.get(url).text, "html.parser")
        if int(soup.h1.text.split(" ")[0]) != year:
            break
        if soup.find("tr", class_ = "oddrow").text == 'No rankings available.':
            year -= 1
            week = 1
            continue
        print '\rScraping Week:', week,"; Year:", year,
        if first:
            headers, data = scrape_table(year, week, soup, True)
        else:
            data = data + scrape_table(year, week, soup)
        week +=1
    print 'Finished!'
    print 'Saving to csv...',
    with open(path, 'wb') as f:
        w = csv.writer(f)
        w.writerow(headers)
        w.writerows(data)
    print 'Finished!'

In [11]:
scrape_AP_Top_25_ESPN()

Scraping Week: 18 ; Year: 2002 Finished!
Saving to csv... Finished!
